In [6]:
%load_ext autotime
import geopandas as gpd
import pandas as pd
import numpy as np
from scipy.spatial import KDTree
from tqdm.auto import tqdm
from glob import glob
from shapely.geometry import LineString, Point
import folium
pd.set_option('display.max_columns', None)

In [7]:
new_shoreline = gpd.GeoDataFrame.from_features([{"type":"Feature","properties":{},"geometry":{"type":"LineString","coordinates":[[173.324218,-41.202358],[173.326406,-41.201228],[173.328466,-41.200001],[173.330784,-41.198806],[173.332801,-41.19787],[173.334603,-41.196837],[173.337049,-41.196223],[173.33971,-41.195513],[173.342414,-41.194479],[173.344774,-41.193349],[173.347092,-41.191928],[173.349967,-41.190443],[173.352284,-41.189248],[173.354816,-41.188214],[173.357821,-41.186858],[173.360353,-41.185824],[173.362885,-41.184371],[173.36473,-41.183111]]}}], crs="EPSG:4326")
new_shoreline

,geometry
0,"LINESTRING (173.32422 -41.20236, 173.32641 -41..."


In [8]:
new_shoreline.explore(tiles="ESRI.WorldImagery")

In [9]:
shorelines = gpd.read_file("shorelines.geojson")
transects = gpd.read_file("transects_extended.geojson")
poly = gpd.read_file("polygons.geojson")

In [10]:
nz_shorelines = shorelines[shorelines.id.str.startswith("nzd")]
latest_siteid = nz_shorelines.id.max()
new_siteid = f"nzd0{int(latest_siteid[4:])+1}"
print(f"Latest siteid is {latest_siteid}, so this new site will be {new_siteid}")
new_shoreline["id"] = new_siteid
new_shoreline

Latest siteid is nzd0563, so this new site will be nzd0564


,geometry,id
0,"LINESTRING (173.32422 -41.20236, 173.32641 -41...",nzd0564


In [11]:
pd.concat((shorelines, new_shoreline)).to_file("shorelines.geojson", driver="GeoJSON")

In [12]:
new_poly = gpd.GeoDataFrame([{"id": new_siteid}], geometry=new_shoreline.to_crs(2193).buffer(100).minimum_rotated_rectangle().to_crs(4326), crs=4326)
new_poly.explore()

In [13]:
pd.concat((poly, new_poly)).to_file("polygons.geojson", driver="GeoJSON")

In [14]:
nz_transects = transects[transects.id.str.startswith("nzd")].to_crs(2193)
nz_transects

,id,site_id,orientation,along_dist,along_dist_norm,beach_slope,cil,ciu,trend,n_points,n_points_nonan,r2_score,mae,mse,rmse,intercept,ERODIBILITY,geometry
95050,nzd0001-0000,nzd0001,359.037136,3197.737936,1.000000,0.050,0.0397,0.0679,-0.048007,194.0,152.0,0.000243,19.220344,557.970123,23.621391,324.525695,None,"LINESTRING (1596662.375 6190263.089, 1596646.6..."
95051,nzd0001-0001,nzd0001,359.037136,3097.737936,0.968728,NaN,NaN,NaN,-0.193058,194.0,166.0,0.004690,17.523830,479.691514,21.901861,329.329626,None,"LINESTRING (1596744.903 6190264.499, 1596729.1..."
95052,nzd0001-0002,nzd0001,359.037136,2997.737936,0.937456,0.060,0.0478,0.0776,-0.280504,194.0,169.0,0.012589,15.783007,369.596582,19.224895,335.326568,None,"LINESTRING (1596827.431 6190265.908, 1596811.6..."
95053,nzd0001-0003,nzd0001,359.037136,2897.737936,0.906184,NaN,NaN,NaN,-0.319317,194.0,171.0,0.020475,14.218951,297.704034,17.254102,343.806416,None,"LINESTRING (1596909.959 6190267.317, 1596894.2..."
95054,nzd0001-0004,nzd0001,359.037136,2797.737936,0.874912,0.055,0.0450,0.0664,-0.385838,194.0,172.0,0.032657,13.279462,274.092958,16.555753,356.283423,None,"LINESTRING (1596992.486 6190268.724, 1596976.7..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
127424,nzd0562-0017,nzd0562,NaN,NaN,NaN,0.055,0.0541,0.0608,0.076617,614.0,436.0,0.002210,8.509662,150.990637,12.287825,211.619988,None,"LINESTRING (1770180.582 5917901.338, 1770542.2..."
127425,nzd0562-0018,nzd0562,NaN,NaN,NaN,0.045,0.0434,0.0461,0.366122,614.0,398.0,0.031111,10.461939,238.631894,15.447715,215.711497,None,"LINESTRING (1770147.813 5917970.658, 1770509.4..."
127426,nzd0563-0000,nzd0563,NaN,NaN,NaN,0.200,0.1756,0.2000,-0.291640,801.0,715.0,0.023067,12.831947,210.266281,14.500561,182.975112,None,"LINESTRING (1939388.56 5638704.895, 1939422.45..."
127427,nzd0563-0001,nzd0563,NaN,NaN,NaN,0.050,0.0323,0.1200,-0.071254,801.0,643.0,0.003210,8.479131,94.816043,9.737353,437.065308,None,"LINESTRING (1939308.842 5638698.136, 1939342.7..."


In [15]:
def create_transects(line, spacing=80, transect_length=400):
    transects = []
    distances = np.arange(0, line.length, spacing)

    for distance in distances:
        # Find point along the average line
        point = line.interpolate(distance)
        
        # Find the direction of the line at this point (tangent direction)
        nearest_point_ahead = line.interpolate(min(distance + 1e-6, line.length))
        direction = np.arctan2(nearest_point_ahead.y - point.y, nearest_point_ahead.x - point.x)
        
        # Rotate 90 degrees (perpendicular) and extend to create a transect
        transect = LineString([
            Point(
                point.x - transect_length / 2 * np.cos(direction + np.pi / 2),
                point.y - transect_length / 2 * np.sin(direction + np.pi / 2)
            ),
            Point(
                point.x + transect_length / 2 * np.cos(direction + np.pi / 2),
                point.y + transect_length / 2 * np.sin(direction + np.pi / 2)
            )
        ])
        
        transects.append(transect)
    
    transects = gpd.GeoDataFrame(geometry=transects, crs=2193).to_crs(4326)
    return transects

new_transects = create_transects(new_shoreline.to_crs(2193).geometry.iloc[0])
display(new_transects)
m = new_transects.explore()
new_shoreline.explore(m=m)
new_poly.boundary.explore(m=m)
gpd.GeoSeries(new_transects.geometry.apply(lambda line: Point(line.coords[0])), crs=new_transects.crs).explore(m=m, color="red", name="transect start")
print("Make sure the origin is inland")
m

,geometry
0,"LINESTRING (173.32556 -41.20384, 173.32287 -41..."
1,"LINESTRING (173.32635 -41.20344, 173.32366 -41..."
2,"LINESTRING (173.32714 -41.20303, 173.32445 -41..."
3,"LINESTRING (173.32805 -41.20254, 173.3251 -41...."
4,"LINESTRING (173.3288 -41.2021, 173.32584 -41.1..."
5,"LINESTRING (173.32955 -41.20165, 173.32659 -41..."
6,"LINESTRING (173.33018 -41.2013, 173.32749 -41...."
7,"LINESTRING (173.33097 -41.20089, 173.32828 -41..."
8,"LINESTRING (173.33176 -41.20048, 173.32907 -41..."
9,"LINESTRING (173.33246 -41.20014, 173.32997 -41..."


Make sure the origin is inland


In [16]:
new_transects["id"] = new_siteid + "-" + new_transects.index.astype(str).str.pad(4, fillchar="0")
new_transects["site_id"] = new_siteid
#new_transects["beach_slope"] = .1
new_transects

,geometry,id,site_id
0,"LINESTRING (173.32556 -41.20384, 173.32287 -41...",nzd0564-0000,nzd0564
1,"LINESTRING (173.32635 -41.20344, 173.32366 -41...",nzd0564-0001,nzd0564
2,"LINESTRING (173.32714 -41.20303, 173.32445 -41...",nzd0564-0002,nzd0564
3,"LINESTRING (173.32805 -41.20254, 173.3251 -41....",nzd0564-0003,nzd0564
4,"LINESTRING (173.3288 -41.2021, 173.32584 -41.1...",nzd0564-0004,nzd0564
5,"LINESTRING (173.32955 -41.20165, 173.32659 -41...",nzd0564-0005,nzd0564
6,"LINESTRING (173.33018 -41.2013, 173.32749 -41....",nzd0564-0006,nzd0564
7,"LINESTRING (173.33097 -41.20089, 173.32828 -41...",nzd0564-0007,nzd0564
8,"LINESTRING (173.33176 -41.20048, 173.32907 -41...",nzd0564-0008,nzd0564
9,"LINESTRING (173.33246 -41.20014, 173.32997 -41...",nzd0564-0009,nzd0564


In [17]:
pd.concat((transects, new_transects)).to_file("transects_extended.geojson", driver="GeoJSON")